In [11]:
# Import libraries
from openai import OpenAI
import os
from google.colab import userdata
import pandas as pd

In [12]:
# Instantiate an OpenAI object, passing in API key
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [28]:
# Read dataset into a dataframe
df = pd.read_csv('drive/MyDrive/Product Reviews.csv')

In [26]:
# Set the display options to show all columns
pd.set_option('display.max_columns', None)

# Set the display width of columns to show full content
pd.set_option('display.max_colwidth', None)

In [29]:
# Get first 5 rows of dataframe
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most.
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar. And it is a tiny mouthful of heaven. Not too chewy, and very flavorful. I highly recommend this yummy treat. If you are familiar with the story of C.S. Lewis' ""The Lion, The Witch, and The Wardrobe"" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch."
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal.
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."


In [30]:
# Subset dataframe to have only "Text" column as that is the column with the full product review.
df = df[['Text']]

In [31]:
# Get no. of rows in the dataframe
df.shape[0]

568454

In [32]:
# Reduce the dataframe to 10 rows to avoid making excessive calls to Open AI API
# which might exhaust current plan
df_undersampled = df.sample(10)

In [40]:
# Define a function named get_sentiment that takes a prompt and an optional model parameter
def get_sentiment(prompt, model="gpt-3.5-turbo"):

    # Define a list of messages, including a system message and a user message
    messages = [
        {"role": "system", "content": "You will be provided with a movie review, and your task is to classify its sentiment and output a one word response: positive, neutral, or negative."},
        {"role": "user", "content": prompt}
    ]

    # Use OpenAI API to generate a chat-based completion response
    response = client.chat.completions.create(
        model=model,  # Specify the GPT model to use (default is "gpt-3.5-turbo")
        messages=messages,  # Pass the list of messages to the OpenAI API
        temperature=0.2,  # Control the randomness of the response (higher values make it more random)
        max_tokens=64,  # Limit the maximum number of tokens in the generated response
        top_p=1  # Set the nucleus sampling probability to 1, meaning it considers the entire probability mass for sampling
    )

    # Return the content of the generated response
    return response.choices[0].message.content

In [41]:
# Create new column "Sentiment" in undersampled dataframe and apply the get_sentiment method
df_undersampled['Sentiment'] = df_undersampled['Text'].apply(get_sentiment)

In [42]:
# View the sentiments of each product review
df_undersampled.reset_index()

,index,Text,Sentiment
0,48680,Definitely try this stuff out. Heat it in the microwave for 10 seconds and its pure heaven. I love it. Tastes amazing!,positive
1,402494,"I saw these made on the TV show ""Unwrapped"", and they sounded like something I'd enjoy. Wrong. These cheese isn't sharp enough and they add way too much salt - it's all I can taste.",negative
2,273218,My wife wanted to try this because she didn't like the donut shop coffee. I was also surpriced to find out that this is a great cup of coffee in the morning and then I use my donut shop for the rest of the day. What we were so happy about was that it shipped in the value pack. That made for a great price for a great cup of coffee.,positive
3,480763,"Season Skinless & Boneless Sardines in Oil IF YOU LIKE SARDINES, TRY THESE LITTLE FISH. YOU MAY WANT TO BUY MORE JUST BECAUSE THEY TASTE GOOD. PLUS ALL THAT NUTRITIOUS STUFF THAT IS GOOD FOR US.",positive
4,286130,"I purchased this product after reading great reviews on here. I really like chocolate / mocha flavors, and was interested to try the raspberry. Got the two boxes, checked the dates (sometimes an issue with K-cups I have found), and eagerly popped one into my brewer. Smell was amazing and I was excited. But when I tasted it, I was so disappointed. Not strong on either the chocolate or raspberry fronts, and the coffee flavor was weak as well. Now I am stuck with 47 cups I don't want. I wish I could recommend these, but I must dissent against the popular reviews. I realize everyone's tastebuds are a bit different, but these are not for me!",negative
5,532846,"I have no idea if this really works, but it tastes good and makes sure I am drinking fluids. I was told that I would need to drink 12 cups of this a day to help promote milk production. The reason I didn't give it 5 stars is because each bag only gives 2 cups worth of tea. Other tea I drink will last me at least 4 cups worth. Other than that no complaints.",neutral
6,283431,Great quality bars! The flavor is exceptionally tasty and you can't beat the price! Would certainly buy again! Note: does contain 96 bars as is advertised!,positive
7,597,"I found these more than 10 years ago in a health food store. After I bought one bag I found myself stopping by that store once a week to get these chips. The New York Cheddar flavor is so good, you will have a hard time not eating a whole bag.",positive
8,519562,"After reading over the reviews for several different coffee brands and flavors, I decided to go with Gloria Jeans Hazelnut and I am so pleased that I did. The price is far less then what I was paying at the stores. The flavor is excellent, the boxing and individual k-cups arrived in perfect condition,(I had read that some did not). I am going to set up for the automatic renewal. What more can I say, it works great for me. Thanks again amazon!!",positive
9,372403,"Thank you Shepherds Garden for a great product! The scriptures are well chosen, thoughtful. It really makes a perfect gift and accompaniment to quiet time (or any time!) As for flavor, compared to celestial seasoning's (non-organic) country peach passion, Shepherds is has soft peach tones and a more earthy flavor. Unless you're not a peach person or avid tea lover you should love it. It's organic too, which is VERY important for tea, and being a white tea it also contains less fluoride if that's a consideration for you. I got 5 boxes directly from their site for my mom, grandmom, sister, and friends this Christmas (great to pair w/ homemade breads or coffeecake!). If you don't think you'll dig peach they have a chai that's truly superior to any chai I've tasted, cranberry is good & unique, and peppermint just rocks (free samples at their website too...). No, they're not flavored with crazy artificial chemicals, sweeteners, non-organic leaves, and colorants. But I LIKE THAT! And for goodness sake they're Bible verse teas! How cool is that?!",positive
